In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split as tts

In [2]:
data = pd.read_excel("D:\\GaTech\\BIOL 8901\\Metabolomic Project\\metabolomic_data.xlsx", sheet_name="1-clean data")
data.rename(columns = {'Unnamed: 0': 'ID'}, inplace = True)

C:\Users\nilav\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [13]:
sample = pd.read_csv("D:\\GaTech\\BIOL 8901\\sample_info.csv")
sample.rename(columns = {'CCLE_Name':'ID'}, inplace=True)
merged_data = data.merge(sample, on='ID')
drug = pd.read_csv('D:\\GaTech\\BIOL 8901\\sanger-dose-response.csv')
drug.rename(columns={'ARXSPAN_ID':'DepMap_ID'}, inplace = True)
working_data = merged_data.merge(drug, on='DepMap_ID')

In [38]:
#drop the columns with NaN in them (not sure if this is the correct way)

wd = working_data.select_dtypes(np.number).dropna(axis=1, how='all')

In [48]:
wd.head(1)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2,IC50_normalized,labels
0,6.112727,6.034198,5.896896,6.000532,5.513618,5.868529,5.977177,5.693074,5.923737,5.641242,...,0.932547,0.653387,-8.901463,0.535007,0.90019,NaN,0.001151,0.927925,0.594693,NR


In [40]:
wd['IC50_normalized'] = np.log2(wd['IC50_PUBLISHED'])

In [42]:
wd[['IC50_normalized']].head(1)

,IC50_normalized
0,0.594693


In [43]:
drug_mean = wd['IC50_normalized'].mean()

In [44]:
#Create labels based on their IC50 response being >< +-0.5 of the mean value as R (responsive) or NR (not responsive)

ic, labels = wd['IC50_normalized'], []
for i in range(len(ic)):
    if ic[i] > drug_mean+0.5:
        labels.append("R")
    elif ic[i] < drug_mean+0.5:
        labels.append("NR")

In [45]:
wd['labels'] = labels

In [47]:
wd.head(1)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2,IC50_normalized,labels
0,6.112727,6.034198,5.896896,6.000532,5.513618,5.868529,5.977177,5.693074,5.923737,5.641242,...,0.932547,0.653387,-8.901463,0.535007,0.90019,NaN,0.001151,0.927925,0.594693,NR


In [50]:
y = wd['labels']
X = wd.select_dtypes('float64')

In [51]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.4, random_state=42)

In [52]:
y_train.head(1)

242923    R
Name: labels, dtype: object

In [53]:
y_test.head(1)

15392    R
Name: labels, dtype: object

In [54]:
X_train.head(1)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,AUC_PUBLISHED,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2,IC50_normalized
242923,6.339798,6.075754,6.670431,5.875871,6.362974,5.833013,5.535336,5.880482,5.922362,6.266865,...,0.950893,0.966275,2.841194,-8.124305,0.853534,0.945736,NaN,0.001115,0.637917,5.381076


In [55]:
X_test.head(1)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,AUC_PUBLISHED,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2,IC50_normalized
15392,5.734549,5.741243,5.726934,5.558014,5.837319,6.174542,5.922278,5.751083,5.778905,5.675493,...,0.868319,0.914686,227.581563,-1.289931,-6.186109,0.872525,NaN,0.000811,0.91381,4.379309


In [56]:
X_train.shape, y_train.shape

((151131, 245), (151131,))

In [57]:
X_test.shape, y_test.shape

((100755, 245), (100755,))

In [58]:
clf = svm.SVC(kernel="linear", C=1).fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').